In [175]:
import numpy as np
import torch
import torchvision
import matplotlib.pyplot as plt
from time import time
from torchvision import datasets, transforms
from torch import nn, optim
import random


In [176]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,)),])
a = open('processed.cleveland.data', 'r')
c = a.readlines()
random.shuffle(c) #shuffle data before using
all_val, all_keys = [], []
for elem in c:
    elem = elem[:-1]
    elem = list(map(float,elem.split(',')))
    all_val.append(elem[:-1])
    all_keys.append(elem[-1])
all_val = np.array(all_val).T

std = []
mean = []

for elem in all_val:
    #print(elem)
    new_elem = np.array(elem)
    #print(np.std(new_elem))
    #print(new_elem.shape)
    std.append(np.std(new_elem))
    mean.append(np.mean(new_elem))
    #elem = [int(elem[i]*10) for i in range(len(elem))] 
for i in range(len(all_val)):
    all_val[i] = [(all_val[i][j] - mean[i])/std[i] for j in range(len(elem))]

all_val = all_val.T

d_val = []
d_keys = []
d = []
t = (len(c) * 0.7)//1
te = len(c) - t
print(t, te)
a = random.sample(range(int(t + te)), int(t))
val, keys, te_val, te_keys = [],[],[],[]
for i in range(len(all_val)):
    if i in a:
        val.append(all_val[i])
        if all_keys[i] > 0:
            keys.append(1)
        else: 
            keys.append(0)
        #n_keys.append(elem[-1])
        if all_keys[i] > 0:
            d_keys.append(all_keys[i] - 1)
            d_val.append(all_val[i])
    else:
        te_val.append(all_val[i])
        te_keys.append(all_keys[i])
        
    #now we want to normalise all values
# rot = [[] for i in range(len(n_val[0]))]
# mean = []
# std = []
# for i in range(len(n_val)):
#     for j in range(len(n_val[i])):
#         rot[j].append(n_val[i][j])
# #print(len(rot[0]))

# #print(rot[0])
# rot = np.array(rot)
# n_val = rot.T     #all of the previous 20 lines normalises our array


# #true-false
# for i in range(len(n_keys)):
#     if n_keys[i] > 0:
#         n_keys[i] = 1
# print(n_keys)


207.0 90.0


In [177]:
#73 - test size
#230 - train size
t = (len(n_val) * 0.7) %1
te = len(n_val) - t
a = random.sample(range(t + te), t)
val, keys, te_val, te_keys = [],[],[],[]
for i in range(len(n_val)):
    if i in a:
        val.append(n_val[i])
        keys.append(n_keys[i])
    else:
        te_val.append(n_val[i])                       #train-test 
        te_keys.append(n_keys[i])

val = np.array(val)
keys = np.array(keys)
#val = val.astype(float)
#keys = keys.astype(float)

te_val = np.array(te_val)
te_keys = np.array(te_keys)
#keys.shape
val.shape

TypeError: 'float' object cannot be interpreted as an integer

In [178]:
# #73 - test size
# #230 - train size
# d_t = (len(dis_val) * 0.7) %1
# d_te = len(dis_val) - t
# a = random.sample(range(d_t + d_te), d_t)
# d_val, d_keys, d_te_val, d_te_keys = [],[],[],[]
# for i in range(len(dis_val)):
#     if i in a:
#         d_val.append(dis_val[i])
#         d_keys.append(dis_keys[i])
#     else:
#         d_te_val.append(dis_val[i])                       #train-test 
#         d_te_keys.append(dis_keys[i])

# d_val = np.array(d_val)
# d_keys = np.array(d_keys)
# #val = val.astype(float)
# #keys = keys.astype(float)

# d_te_val = np.array(d_te_val)
# d_te_keys = np.array(d_te_keys)
# #keys.shape
# val.shape

In [179]:
input_size = 13
hidden_sizes = [256, 128] #was128
output_size = 2

model = nn.Sequential(nn.Linear(input_size, hidden_sizes[0]),
                      nn.ReLU(),
                      nn.Linear(hidden_sizes[0], hidden_sizes[1]),
                      nn.ReLU(),
                      nn.Linear(hidden_sizes[1], output_size),
                      nn.LogSoftmax(dim=1))
print(model) #model with 3 layers, first 2 with relu, last with logistic curve

Sequential(
  (0): Linear(in_features=13, out_features=256, bias=True)
  (1): ReLU()
  (2): Linear(in_features=256, out_features=128, bias=True)
  (3): ReLU()
  (4): Linear(in_features=128, out_features=2, bias=True)
  (5): LogSoftmax()
)


In [180]:
input_size = 13
hidden_sizes = [256, 128] #was128
output_size = 4

d_model = nn.Sequential(nn.Linear(input_size, hidden_sizes[0]),
                      nn.ReLU(),
                      nn.Linear(hidden_sizes[0], hidden_sizes[1]),
                      nn.ReLU(),
                      nn.Linear(hidden_sizes[1], output_size),
                      nn.LogSoftmax(dim=1))
print(d_model) #model with 3 layers, first 2 with relu, last with logistic curve

Sequential(
  (0): Linear(in_features=13, out_features=256, bias=True)
  (1): ReLU()
  (2): Linear(in_features=256, out_features=128, bias=True)
  (3): ReLU()
  (4): Linear(in_features=128, out_features=4, bias=True)
  (5): LogSoftmax()
)


In [181]:
criterion = nn.NLLLoss() #MSELoss() 
#images, labels = next(iter(trainloader))
#images = images.view(images.shape[0], -1)

#logps = model(images) #log probabilities
#loss = criterion(logps, labels) #calculate the NLL loss

In [182]:
keys[0]

1

In [183]:
optimizer = optim.SGD(model.parameters(), lr=0.005, momentum=0.9)
time0 = time()
epochs = 30
for e in range(epochs):
    running_loss = 0
    for i in range(len(val)):
        # Training pass
        optimizer.zero_grad()
        temp_1 = [val[i]]
        temp0 = np.array(temp_1)
        temp = torch.from_numpy(temp0)
        temp1 = temp.float()
        #print(temp1)
        output = model(temp1)    #DOES NOT WORK. MODEL RETURNS NAN TENSOR
        #print(output)
        k = [keys[i]]
        k = np.array(k)
        k = torch.from_numpy(k)
        k = k.long()
        #print(output.shape)
        #print(k.shape)
        #print('result', e, output)
        #print(k)
        loss = criterion(output, k)
        #print(loss)
        
        #This is where the model learns by backpropagating
        loss.backward()
        
        #And optimizes its weights here
        optimizer.step()
        
        #print(loss.item())
        running_loss += loss.item()
    else:
        print("Epoch {} - Training loss: {}".format(e, running_loss/len(val)))

print("\nTraining Time (in minutes) =",(time()-time0)/60)

Epoch 0 - Training loss: 0.47633558931096304
Epoch 1 - Training loss: 0.38341724949542016
Epoch 2 - Training loss: 0.34652491575957245
Epoch 3 - Training loss: 0.31303071337481525
Epoch 4 - Training loss: 0.27776940920326154
Epoch 5 - Training loss: 0.2386239735093384
Epoch 6 - Training loss: 0.1987337485960751
Epoch 7 - Training loss: 0.14483016017170883
Epoch 8 - Training loss: 0.25697512233852193
Epoch 9 - Training loss: 0.22036494513445187
Epoch 10 - Training loss: 0.16510987051958453
Epoch 11 - Training loss: 0.07984425564911216
Epoch 12 - Training loss: 0.05534398595696324
Epoch 13 - Training loss: 0.033220136831430824
Epoch 14 - Training loss: 0.023261321930241635
Epoch 15 - Training loss: 0.017709181021697593
Epoch 16 - Training loss: 0.014509133698282307
Epoch 17 - Training loss: 0.012630663723932607
Epoch 18 - Training loss: 0.010794019750365029
Epoch 19 - Training loss: 0.009188434865143967
Epoch 20 - Training loss: 0.007982969051285533
Epoch 21 - Training loss: 0.0069278325

In [184]:
d_optimizer = optim.SGD(model.parameters(), lr=0.005, momentum=0.9)
time0 = time()
epochs = 30
for e in range(epochs):
    running_loss = 0
    for i in range(len(d_val)):
        # Training pass
        optimizer.zero_grad()
        temp_1 = [d_val[i]]
        temp0 = np.array(temp_1)
        temp = torch.from_numpy(temp0)
        temp1 = temp.float()
        #print(temp1)
        output = d_model(temp1)    #DOES NOT WORK. MODEL RETURNS NAN TENSOR
        #print(output)
        k = [d_keys[i]]
        k = np.array(k)
        k = torch.from_numpy(k)
        k = k.long()
        #print(output.shape)
        #print(k.shape)
        #print('result', e, output)
        #print(k)
        #print(output)
        #print(k)
        loss = criterion(output, k)
        #print(loss)
        
        #This is where the model learns by backpropagating
        loss.backward()
        
        #And optimizes its weights here
        optimizer.step()
        
        #print(loss.item())
        running_loss += loss.item()
    else:
        print("Epoch {} - Training loss: {}".format(e, running_loss/len(val)))

print("\nTraining Time (in minutes) =",(time()-time0)/60)

Epoch 0 - Training loss: 0.6487609366863822
Epoch 1 - Training loss: 0.6487609366863822
Epoch 2 - Training loss: 0.6487609366863822
Epoch 3 - Training loss: 0.6487609366863822
Epoch 4 - Training loss: 0.6487609366863822
Epoch 5 - Training loss: 0.6487609366863822
Epoch 6 - Training loss: 0.6487609366863822
Epoch 7 - Training loss: 0.6487609366863822
Epoch 8 - Training loss: 0.6487609366863822
Epoch 9 - Training loss: 0.6487609366863822
Epoch 10 - Training loss: 0.6487609366863822
Epoch 11 - Training loss: 0.6487609366863822
Epoch 12 - Training loss: 0.6487609366863822
Epoch 13 - Training loss: 0.6487609366863822
Epoch 14 - Training loss: 0.6487609366863822
Epoch 15 - Training loss: 0.6487609366863822
Epoch 16 - Training loss: 0.6487609366863822
Epoch 17 - Training loss: 0.6487609366863822
Epoch 18 - Training loss: 0.6487609366863822
Epoch 19 - Training loss: 0.6487609366863822
Epoch 20 - Training loss: 0.6487609366863822
Epoch 21 - Training loss: 0.6487609366863822
Epoch 22 - Training 

In [185]:
correct_count, all_count = 0, 0    

for i in range(len(te_val)):
    temp_1 = [te_val[i]]
    temp0 = np.array(temp_1)
    temp = torch.from_numpy(temp0)
    temp1 = temp.float()
    with torch.no_grad():
        logps = model(temp1)
    ps = torch.exp(logps)
    probab = list(ps.numpy()[0])
    pred_label = probab.index(max(probab))
    #print(pred_label)
    if pred_label == 0:
        fin_label = 0
    else:
        with torch.no_grad():
            d_logps = d_model(temp1) #set temp1 corr
        d_ps = torch.exp(d_logps)
        d_probab = list(d_ps.numpy()[0])
        d_pred_label = d_probab.index(max(d_probab))
        fin_label = d_pred_label + 1
    
    true_label = te_keys[i]
    if(true_label == fin_label):
        correct_count += 1      
    all_count += 1

print("Number Of Images Tested =", all_count)
print("\nModel Accuracy =", (correct_count/all_count))

Number Of Images Tested = 90

Model Accuracy = 0.5333333333333333


In [186]:
# correct_count, all_count = 0, 0    
# norm_c, norm_true = 0,0
# dis_c, dis_true = 0,0

# for i in range(len(te_val)):
#     temp_1 = [te_val[i]]
#     temp0 = np.array(temp_1)
#     temp = torch.from_numpy(temp0)
#     temp1 = temp.float()
#     with torch.no_grad():
#         logps = model(temp1)
#     ps = torch.exp(logps)
#     probab = list(ps.numpy()[0])
#     pred_label = probab.index(max(probab))
#     true_label = te_keys[i]
#     if(true_label == pred_label):
#         correct_count += 1
#         if true_label == 0:
#             norm_true += 1
#         else:
#             dis_true += 1
#     if true_label == 0:
#         norm_c += 1
#     else:
#         dis_c += 1        
#     all_count += 1

# print("Number Of Images Tested =", all_count)
# print("\nModel Accuracy =", (correct_count/all_count))

In [ ]:
#true-false 80 percent
# print("Number Of norm =", norm_c)
# print("\nModel Accuracy norm=", (norm_true/norm_c))
# print("Number Of dis =", dis_c)
# print("\nModel Accuracy dis=", (dis_true/dis_c))

In [ ]:
torch.save(model, './my_mnist_model.pt') 